TH Nürnberg - Neural Network Translator - Christoph Brandl, Philipp Grandeit

# PoC: Creation of a Frontend for CNTK Framework

This jupyter notebook provides an exemplary implementation of the extraction of information from a neural network model created with the CNTK framework.

In contrast to the rest of this repository, this notebook requires Python 3.6 in order to install and use the cntk module, because the cntk module is not available for Python 3.7. To install a Python 3.6 environment, a file called "nnt_36_cntk_environment.yml" is provided in the setup directory. It is possible to create a conda environment with all required modules using the provided environment template. Please note that all other resources of this repository are written and tested with Python 3.7 and, therefore, Python 3.7 is the prefered version. Python 3.6 is only used for this specific proof of concept.

In [ ]:
import cntk as C
import numpy as np

# Building the Network

A feed-forward network with multiple dense layers is created.

In [ ]:
input_dim = 8
num_output_classes = 1
mysamplesize = 64
num_hidden_layers = 1
hidden_layers_dim = 8

input = C.input_variable(input_dim)
label = C.input_variable(num_output_classes)

In [ ]:
def create_model(features):
    with C.layers.default_options(init=C.layers.glorot_uniform(), activation=C.sigmoid):
        h = features
        for _ in range(num_hidden_layers):
            h = C.layers.Dense(hidden_layers_dim, activation=C.sigmoid)(h)
        last_layer = C.layers.Dense(num_output_classes, activation = None)

        return last_layer(h)

In [ ]:
model = create_model(input)

# Save the Network Model

The functionality of saving notebooks with the CNTK framework is available.

In [ ]:
model.save('cntk_diabetes.model')

# Load the Network Model

The functionality for loading notebooks with the CNTK framework is also available

In [ ]:
from cntk.ops.functions import load_model

z = load_model("cntk_diabetes.model")

# Read Network Structure and Information from the loaded Model

Now the information saved in the model can be read. First we have a look at the structure of the loaded model containing all needed information.

The information for each layer can be accessed by looping through the layers. Thus, it would possible to create a json object which complies to the required structure of the json intermediate format for the Neural Network Translator. This shows that it is possible to translate a model created with the CNTK framework into the used intermediate format and, therefore, it is possible to create a frontend plugin which can process models created with the CNTK framework.

In [ ]:
layers = C.logging.graph.depth_first_search(
            model, (lambda x : type(x) == C.Function and x.is_block) , depth = 0)
for layer in layers:
    print(layer.op_name) #Layer type, e.g. dense
    print(layer.is_composite)
    print(layer.arguments) 
    print(layer.constants)
    print(layer.parameters)
    print(layer.attributes)
    print('-----')

Also the weights and biases can be read via the parameters field of each layer.

In [ ]:
parameters = z.parameters
for parameter in parameters:
    print(parameter.name, parameter.shape, "\n", parameter.value) 

### END